In [20]:
!pip install -U -q mlx-lm
!pip install -U -q langchain_community langchain-cohere langchain
!pip install -U -q pypdf faiss-cpu python-dotenv defusedxml tokenizers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chat-with-mlx 0.1.11 requires langchain==0.1.16, but you have langchain 0.2.5 which is incompatible.
chat-with-mlx 0.1.11 requires langchain-core==0.1.45, but you have langchain-core 0.2.7 which is incompatible.
transformers 4.41.0.dev0 requires tokenizers<0.20,>=0.19, but you have tokenizers 0.15.2 which is incompatible.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cohere 5.5.7 requires tokenizers<0.16,>=0.15, but you have tokenizers 0.19.1 which is incompatible.


In [1]:
from operator import itemgetter

from langchain_community.chat_models.mlx import ChatMLX
from langchain_community.llms.mlx_pipeline import MLXPipeline

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.output_parsers import XMLOutputParser
from langchain_core.output_parsers import StrOutputParser


from langchain.schema import (
    HumanMessage,
    SystemMessage,
    AIMessage
)

# Router

In [2]:
router_llm = MLXPipeline.from_model_id(
    "mlx-community/Mistral-7B-Instruct-v0.3-4bit",
    pipeline_kwargs={"max_tokens": 200, "temp": 0.1},
)

tools = [
    {
        "type": "function",
        "function": {
            "name": "retriever",
            "description": "Useful for retrieving factual documents and information from a database or API about a user's request to answer their queries.",
            "parameters": {
                "type": "object",
                "properties": {
                    "user_query": {
                        "type": "string",
                        "description": "The query to retrieve and ground information for."
                    },

                },
                "required": ["user_query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "direct_response",
            "description": "Useful for providing a direct response without retrieving additional information.",
            "parameters": {
                "type": "object",
                "properties": {
                    "user_query": {
                        "type": "string",
                    }
                },
                "required": ["user_query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "vision",
            "description": "Useful for analysing images.",
            "parameters": {
                "type": "object",
                "properties": {
                    "user_query": {
                        "type": "string",
                    },
                    "image_path": {
                        "type": "string",
                    }
                },
                "required": ["user_query", "image_path"]
            }
        }
    }
]


Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [3]:
template = """ You are Mistral with function-calling supported. You are provided with function signatures within <tools></tools> XML tags.
    You may call one function to assist with the user query. Don't make assumptions about what values to plug into functions.
    Here are the available tools:
    <tools>
    {tools}
    </tools>

    For each function call, return a XML object with the function name and arguments within <tool_call></tool_call> XML tags as follows:
    <tool_call>
        <function_name></function_name>
        <arguments>
            <name></name>
            <value></value>
            <image_path></image_path>
        </arguments>
    </tool_call>

    If the user question requires factual answers, use the retriever tool only.

    <tool_call>
        <function_name>retriever</function_name>
        <arguments>
            <name></name>
            <value></value>
        </arguments>
    </tool_call>

    Please respond using XML ONLY.
    Question: {question}
"""

In [4]:
prompt = ChatPromptTemplate.from_template(template)
router = ChatMLX(llm=router_llm)

router_chain = (
    {"tools": itemgetter("tools"), "question": RunnablePassthrough(), "image": RunnablePassthrough()}
    | prompt
    | router
    | XMLOutputParser()
)

In [5]:
router_output = router_chain.invoke({"tools": tools, "question": "Hi, how are you??"})
router_output

{'tool_call': [{'function_name': 'direct_response'},
  {'arguments': [{'name': 'user_query'}, {'value': 'Hi, how are you??'}]}]}

In [6]:
router_output = router_chain.invoke({"tools": tools, "question": "Caption this image './assets/training_pipelines.jpg'"})
router_output

{'tool_call': [{'function_name': 'vision'},
  {'arguments': [{'name': 'user_query'},
    {'value': "Caption this image './assets/training_pipelines.jpg'"},
    {'name': 'image_path'},
    {'value': './assets/training_pipelines.jpg'}]}]}

In [7]:
router_output = router_chain.invoke({"tools": tools, "question": "Explain how Gemma model works?", "image": ""})
router_output

{'tool_call': [{'function_name': 'retriever'},
  {'arguments': [{'name': 'user_query'},
    {'value': 'Explain how Gemma model works'}]}]}

# Retriever

In [8]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

def load_file(file_name, file_type):
    loader = PyPDFLoader(f"./assets/{file_name}.{file_type}")

    text_splitter = RecursiveCharacterTextSplitter(
        # Set a really small chunk size, just to show.
        chunk_size=5000,
        chunk_overlap=20,
        length_function=len,
        is_separator_regex=False,
    )
    return loader.load_and_split(text_splitter)


documents = load_file('gemma-report', 'pdf')
vectorstore = FAISS.from_documents(
    documents, embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
)
retriever = vectorstore.as_retriever()

/opt/homebrew/Caskroom/miniconda/base/envs/mlx_code/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
retriever.invoke("Gemma")

[Document(page_content='Gemma: Open Models Based on Gemini Research and Technology\nContributions and Acknowledgments\nCore Contributors\nThomas Mesnard\nCassidy Hardin\nRobert Dadashi\nSurya Bhupatiraju\nShreya Pathak\nLaurent Sifre\nMorgane Rivière\nMihir Sanjay Kale\nJuliette Love\nPouya Tafti\nLéonard Hussenot\nContributors\nAakanksha Chowdhery\nAdam Roberts\nAditya Barua\nAlex Botev\nAlex Castro-Ros\nAmbrose Slone\nAmélie Héliou\nAndrea Tacchetti\nAnna Bulanova\nAntonia Paterson\nBeth Tsai\nBobak Shahriari\nCharline Le Lan\nChristopher Choquette\nClément Crepy\nDaniel Cer\nDaphne Ippolito\nDavid Reid\nElena Buchatskaya\nEric Ni\nEric Noland\nGeng Yan\nGeorge Tucker\nGeorge-Christian Muraru\nGrigory Rozhdestvenskiy\nHenryk Michalewski\nIan Tenney\nIvan Grishchenko\nJacob Austin\nJames Keeling\nJane Labanowski\nJean-Baptiste Lespiau\nJeff Stanway\nJenny BrennanJeremy Chen\nJohan Ferret\nJustin Chiu\nJustin Mao-Jones\nKatherine Lee\nKathy Yu\nKatie Millican\nLars Lowe Sjoesund\nLisa 

In [9]:
writer_llm = MLXPipeline.from_model_id(
    "mlx-community/Qwen2-0.5B-Instruct",
    pipeline_kwargs={"max_tokens": 100, "temp": 0.7},
)
writer = ChatMLX(llm=writer_llm)

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Tool calling + RAG

In [10]:
from mlx_vlm.utils import load, generate, load_image_processor
from mlx_vlm.prompt_utils import get_message_json

model_path = "mlx-community/deepseek-vl-7b-chat-4bit"
vision_model, processor = load(model_path)
image_processor = load_image_processor(model_path)


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

In [11]:
def format_docs(docs):
    formatted_docs = []
    for i, doc in enumerate(docs):
        formatted_doc = f"<doc id='{i}'>{doc.page_content}</doc>"
        formatted_docs.append(formatted_doc)
    return "\n".join(formatted_docs)


def create_retriever_chain():
    template = """Answer the question based only on the following context:
            {context}

            Question: {question}
        """
    prompt = ChatPromptTemplate.from_template(template)

    chain = (
        {
            "context": RunnablePassthrough(),
            "question": RunnablePassthrough(),
        }
        | prompt
        | writer
        | StrOutputParser()
    )

    return chain


def tool_call(function_name, arguments):
    question = arguments[-1]['value']
    if function_name == 'get_weather':

        return "The weather is good!"

    if function_name == 'retriever':
        retriever_chain = create_retriever_chain()
        docs = format_docs(retriever.invoke(question))
        output = retriever_chain.invoke({"context": docs, "question": question})
        return output

    if function_name == 'vision':
        question = arguments[1]['value']
        image_path = arguments[-1]['value']
        print(question, image_path)
        prompt = processor.apply_chat_template(
            [get_message_json(vision_model.config.model_type, question)],
            tokenize=False,
            add_generation_prompt=True,
        )
        output = generate(
            vision_model,
            processor,
            image_path,
            prompt,
            image_processor,
            temp=0.7,
            max_tokens=100,
            verbose=False
        )
        return output

    else:
        messages = [
            HumanMessage(
                content=question
            ),
        ]
        output = writer.invoke(messages)

        return output

In [12]:
function_name= router_output["tool_call"][0]["function_name"]
args = router_output["tool_call"][-1]["arguments"]
tool_call_out = tool_call(function_name, args)

In [13]:
print("Tool:", function_name)
print("Question:", args[-1]["value"])
print("Answer:",tool_call_out)

Tool: retriever
Question: Explain how Gemma model works
Answer: The Gemma model is a large language model (LLM) developed by Google. It is a Transformer-based model that is trained on a context length of 8192 tokens. The model is based on the Transformer decoder and includes a feedforward hidden layer with 32768 filters and 49152 heads. The vocabulary size is 256128, and the number of KV heads is 16. The model has 256


In [15]:
from pprint import pprint

def rag_pipeline(question):
    router_output = router_chain.invoke({"tools": tools, "question": question})
    function_name= router_output["tool_call"][0]["function_name"]
    args = router_output["tool_call"][-1]["arguments"]
    tool_call_out = tool_call(function_name, args)
    print("Tool:", function_name)
    print("Question:", args[-1]["value"])
    print("=============\nAnswer:")

    pprint(tool_call_out)


## RAG

In [17]:
rag_pipeline("How many sizes does gemma have?")

Tool: retriever
Question: How many sizes does Gemma have?
Answer:
'Gemma has a total of 6 sizes.'


## RAG + VISION

In [18]:
rag_pipeline("Describe each stage of this image: './assets/training_pipelines.jpg'")

Describe each stage of this image: './assets/training_pipelines.jpg' ./assets/training_pipelines.jpg
Tool: vision
Question: ./assets/training_pipelines.jpg
Answer:
('The 1: Training VL Adapter\n'
 'In this stage, the VL Adapter is trained using a mixture of image-text pairs '
 'and pure language sequences. The model, DeepSeek LM, is responsible for '
 'processing the input data and generating the outputs required for the '
 'subsequent stages.\n'
 '\n'
 'Stage 2: Joint VL Pro-training\n'
 'In this stage, the VL Adapter is jointly pre-trained with the '
 'vision-and-language model, DeepSeek ViT. The training process involves using '
 'a mixture of image')
